In [4]:
from pyspark.sql.functions import mean, udf, col, round
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import datetime
print(datetime.datetime.now())
%matplotlib inline

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

2019-05-29 15:52:05.649872


## import csv files into spark dataframes
Note: both files contain records from all 5 boroughs

In [5]:
df1 = spark.read.csv('data/pluto_19v1.csv', header=True)
df2 = spark.read.csv('data/pluto_18v2_1.csv', header=True)

In [6]:
df1 = df1.select([col(A).alias(A.lower()) for A in df1.schema.names])
df2 = df2.select([col(A).alias(A.lower()) for A in df2.schema.names])

In [7]:
double_columns = ['bldgarea', 'facilfar',
                  'residfar', 'commfar', 'numbldgs', 'numfloors', 'bldgdepth', 
                  'bldgfront', 'lotdepth', 'lotfront', 
                  'exempttot', 'exemptland',  'assessland', 'assesstot', 'builtfar']

In [8]:
cols = df2.columns

In [9]:
df1 = df1.select(cols)
df2 = df2.select(cols)

## Type Conversion

In [10]:
for A in double_columns: 
    df1 = df1.withColumn(A, round(col(A).cast(DoubleType()), 2))
    df2 = df2.withColumn(A, round(col(A).cast(DoubleType()), 2))

## Change column names and joining dataframes
* since we are doing column comparison, we need to the column names of one df to avoid confusion
* in this case, we add "_1"_ to the name, e.g. "BBL" ==> "BBL_1"
* here we are doing an inner join because we only care about the BBL's that apeared in both dataframes

In [11]:
df1 = df1.select([col(A).alias(A+'_1') for A in df1.schema.names])

In [12]:
df1_names = df1.schema.names
df2_names = df2.schema.names
colnames = zip(df1_names, df2_names)

In [13]:
df = df2.join(df1, df2['bbl'] == df1['bbl_1'])
df0 = df

## Inspect top differences

In [14]:
@udf
def diff(col1,col2):
    try:
        return abs(col2-col1)
    except: 
        pass

In [15]:
spark.sql('set spark.sql.caseSensitive=true')
targets = ['unitsres','lotarea','bldgarea',
             'comarea','resarea',
             'officearea','retailarea',
             'garagearea','strgearea',
             'factryarea','otherarea']
for A in targets:
    df0 = df0.withColumn(A+'_diff', diff(col(A).cast(DoubleType()),col(A+'_1').cast(DoubleType())))

In [16]:
df0.select('bbl', col('unitsres_diff').cast(DoubleType())).orderBy('unitsres_diff', ascending=False).show(5)

+----------+-------------+
|       bbl|unitsres_diff|
+----------+-------------+
|3002390016|       1322.0|
|4004370020|        802.0|
|4004370011|        802.0|
|4004370015|        802.0|
|4004370008|        802.0|
+----------+-------------+
only showing top 5 rows



In [17]:
df0.select('bbl', col('comarea_diff').cast(DoubleType())).orderBy('comarea_diff', ascending=False).show(5)

+----------+------------+
|       bbl|comarea_diff|
+----------+------------+
|1000580001|   5887499.0|
|4009260001|   1401784.0|
|1018860001|   1269723.0|
|1006720001|   1062349.0|
|1006730001|    795335.0|
+----------+------------+
only showing top 5 rows



In [18]:
df0.select('bbl', col('resarea_diff').cast(DoubleType())).orderBy('resarea_diff', ascending=False).show(5)

+----------+------------+
|       bbl|resarea_diff|
+----------+------------+
|1010517502|   5343460.0|
|1000940001|   1813255.0|
|1000160100|   1669272.0|
|4163500400|   1599900.0|
|3044520085|   1548030.0|
+----------+------------+
only showing top 5 rows



In [19]:
df0.select('bbl', col('officearea_diff').cast(DoubleType())).orderBy('officearea_diff', ascending=False).show(5)

+----------+---------------+
|       bbl|officearea_diff|
+----------+---------------+
|1007390001|       676250.0|
|2044270029|       382935.0|
|1009220046|       331459.0|
|4096950014|       321024.0|
|1013100001|       299750.0|
+----------+---------------+
only showing top 5 rows



In [20]:
df0.select('bbl', col('retailarea_diff').cast(DoubleType()))\
    .orderBy('retailarea_diff', ascending=False).show(5)

+----------+---------------+
|       bbl|retailarea_diff|
+----------+---------------+
|5024000180|       574842.0|
|5001020001|       510819.0|
|1006620003|       211000.0|
|1011020029|       189534.0|
|1005107502|       188149.0|
+----------+---------------+
only showing top 5 rows



In [21]:
df0.select('bbl', col('garagearea_diff').cast(DoubleType()))\
    .orderBy('garagearea_diff', ascending=False).show(5)

+----------+---------------+
|       bbl|garagearea_diff|
+----------+---------------+
|4017857504|       630358.0|
|2051350210|       296715.0|
|4049787502|       271294.0|
|3001570001|       248600.0|
|1000180100|       188772.0|
+----------+---------------+
only showing top 5 rows



In [22]:
df0.select('bbl', col('factryarea_diff').cast(DoubleType()))\
    .orderBy('factryarea_diff', ascending=False).show(5)

+----------+---------------+
|       bbl|factryarea_diff|
+----------+---------------+
|1006730001|       795335.0|
|4026490113|       583890.0|
|3081490300|       373403.0|
|2027710045|       361343.0|
|1008397501|       311886.0|
+----------+---------------+
only showing top 5 rows



In [23]:
df0.select('bbl', col('otherarea_diff').cast(DoubleType()))\
    .orderBy('otherarea_diff', ascending=False).show(5)

+----------+--------------+
|       bbl|otherarea_diff|
+----------+--------------+
|1007390001|      549853.0|
|5001020001|      510819.0|
|3078810033|      480000.0|
|1000237501|      479973.0|
|3006440001|      427123.0|
+----------+--------------+
only showing top 5 rows

